In [1]:
# -*- coding: utf-8 -*-
import time
import os
import tensorflow as tf
import numpy as np
%run Bi_LSTM.py
%run Word2Vec.py
import pandas as pd

C:\Users\dongc\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# 여기부터 하면 됨

In [2]:
from tqdm import tqdm_notebook
from eunjeon import Mecab
from konlpy.tag import Okt

In [3]:
mecab = Mecab()
okt = Okt()
W2V = Word2Vec()

In [4]:
df = pd.read_csv('../data/자소서_균등.csv',index_col=0, engine='python', encoding='utf8')

## 데이터 섞기

In [5]:
df = df.reset_index().drop(columns={'index'})

In [6]:
rows = np.random.choice(df.index.values, len(df), replace=False)
sampled_df = df.loc[rows]

In [7]:
df = sampled_df

In [8]:
dataset = int(len(df) * 0.7)

In [9]:
df.iloc[:dataset].groupby('label').agg('count')

,c_origin,category,jss
label,,,
0.0,2194,2194,2194
1.0,2198,2198,2198
2.0,2350,2350,2350
3.0,2175,2175,2175


In [10]:
df.head()

,c_origin,category,jss,label
10457,서비스,기타,대학시절 활발하고 다양한 활동을 통하여 넓은 인맥을 얻었습니다. 어느 한쪽에도 치우...,0.0
5294,응용프로그래머,응용프로그래밍,[어제와 똑같이 살면서 다른 미래를 기대하는 것은 병에 걸린 것과 같다.]\n\n아...,3.0
7624,금융영업,재무·금융,[은행은 해우소(解憂所)]\n은행을 해우소(화장실)라고 표현하는 것이 지저분할 수 ...,2.0
6313,채권·보험·보상·심사,재무·금융,"일단 인턴으로 들어가 일을 하는 것이니만큼, 기본에 충실 하는 사람이 되겠습니다.\...",2.0
5589,웹프로그래머,응용프로그래밍,전자공학과에서 기본적인 프로그래밍을 배웠습니다. C언어와 JAVA가 대표적이었는데 ...,3.0


In [10]:
data = []
for txt in tqdm_notebook(df.jss):
    data.append(mecab.morphs(txt))

In [11]:
labels = df.label

# 워드 임베딩

In [12]:
config = tf.ConfigProto()
tf.reset_default_graph()
W2V = Word2Vec()

In [13]:
dataset = int(len(data)*0.7)

In [14]:
# 70% train, 30% test
train_X = data[:dataset]
train_Y = labels[:dataset]
test_X = data[dataset:]
test_Y = labels[dataset:]

In [15]:
#nb_classes = len(df.category.unique())

In [16]:
nb_classes = len(df.label.unique())

In [17]:
train_X_ = W2V.Convert2Vec("../data/word2vec_mecab_morphs2.model",train_X)
test_X_ = W2V.Convert2Vec("../data/word2vec_mecab_morphs2.model",test_X)

In [18]:
train_Y_ = W2V.One_hot(train_Y)
test_Y_ = W2V.One_hot(test_Y)

# 파라미터

In [19]:
Batch_size = 32
Total_size = len(train_X)
Vector_size = 300 # 모델의 워드벡터 사이즈
train_seq_length = [len(x) for x in train_X]
test_seq_length = [len(x) for x in test_X]
Maxseq_length = max(max(train_seq_length), max(test_seq_length))
learning_rate = 0.001
lstm_units = 128
num_class = nb_classes
training_epochs = 3
print(Maxseq_length)

2467


# 그래프 만들기

In [20]:
X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size], name = 'X')
Y = tf.placeholder(tf.float32, shape = [None, num_class], name = 'Y')
seq_len = tf.placeholder(tf.int32, shape = [None])
keep_prob = tf.placeholder(tf.float32, shape = None)

In [21]:
BiLSTM = Bi_LSTM(lstm_units, num_class, keep_prob)

In [22]:
with tf.variable_scope("loss", reuse = tf.AUTO_REUSE):
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate)
prediction = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [23]:
total_batch = int(len(train_X) / Batch_size)
test_batch = int(len(test_X) / Batch_size)

In [24]:
modelName = "./BiLSTM_mecab_morphs_eqaul_1/BiLSTM_model.ckpt"
saver = tf.train.Saver()

train_acc = []
train_loss = []
test_acc = []
test_loss = []

In [25]:
# 모델 불러오기
saver = tf.train.Saver()
init = tf.global_variables_initializer()
modelName = "BiLSTM_mecab_morphs_eqaul_1/BiLSTM_model.ckpt"

sess = tf.Session(config=config)
sess.run(init)
start_time = time.time()
saver.restore(sess, modelName)
train_writer = tf.summary.FileWriter('./BiLSTM_mecab_morphs_eqaul_1', sess.graph)
merged = BiLSTM.graph_build()

INFO:tensorflow:Restoring parameters from BiLSTM_mecab_morphs_eqaul_1/BiLSTM_model.ckpt


# 세션 런

In [26]:
sess = tf.Session(config=config)
init = tf.global_variables_initializer()
start_time = time.time()
sess.run(init)
train_writer = tf.summary.FileWriter('./BiLSTM_mecab_morphs_eqaul_1', sess.graph)
merged = BiLSTM.graph_build()

In [26]:
for epoch in range(training_epochs):  
#for epoch in range(5,8):

    avg_acc, avg_loss = 0. , 0.
    mask = np.random.permutation(len(train_X_))
    train_X_ = train_X_[mask]
    train_Y_ = train_Y_[mask]

    for step in tqdm_notebook(range(total_batch)):

        train_batch_X = train_X_[step*Batch_size : step*Batch_size+Batch_size]
        train_batch_Y = train_Y_[step*Batch_size : step*Batch_size+Batch_size]
        batch_seq_length = train_seq_length[step*Batch_size : step*Batch_size+Batch_size]

        train_batch_X = W2V.Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)

        sess.run(optimizer, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
        # Compute average loss
        loss_ = sess.run(loss, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length,
                                          keep_prob : 0.75})
        avg_loss += loss_ / total_batch

        acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length,
                                             keep_prob : 0.75})
        avg_acc += acc / total_batch
        print("epoch : {:02d} step : {:04d} loss = {:.6f} accuracy= {:.6f}".format(epoch+1, step+1, loss_, acc))

    summary = sess.run(merged, feed_dict = {BiLSTM.loss : avg_loss, BiLSTM.acc : avg_acc})       
    train_writer.add_summary(summary, epoch)

    t_avg_acc, t_avg_loss = 0. , 0.
    print(avg_loss, avg_acc)
    print("Test cases, could take few minutes")
    
    for step in tqdm_notebook(range(test_batch)):

        test_batch_X = test_X_[step*Batch_size : step*Batch_size+Batch_size]
        test_batch_Y = test_Y_[step*Batch_size : step*Batch_size+Batch_size]
        batch_seq_length = test_seq_length[step*Batch_size : step*Batch_size+Batch_size]

        test_batch_X = W2V.Zero_padding(test_batch_X, Batch_size, Maxseq_length, Vector_size)

        # Compute average loss
        loss2 = sess.run(loss, feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length,
                                          keep_prob : 1.0})
        t_avg_loss += loss2 / test_batch

        t_acc = sess.run(accuracy , feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length,
                                               keep_prob : 1.0})
        t_avg_acc += t_acc / test_batch

    print("<Train> Loss = {:.6f} Accuracy = {:.6f}".format(avg_loss, avg_acc))
    print("<Test> Loss = {:.6f} Accuracy = {:.6f}".format(t_avg_loss, t_avg_acc))
    train_loss.append(avg_loss)
    train_acc.append(avg_acc)
    test_loss.append(t_avg_loss)
    test_acc.append(t_avg_acc)

epoch : 01 step : 0001 loss = 0.922414 accuracy= 0.750000
epoch : 01 step : 0002 loss = 0.785385 accuracy= 0.718750
epoch : 01 step : 0003 loss = 0.463357 accuracy= 0.875000
epoch : 01 step : 0004 loss = 0.660965 accuracy= 0.781250
epoch : 01 step : 0005 loss = 0.498948 accuracy= 0.718750
epoch : 01 step : 0006 loss = 0.592999 accuracy= 0.812500
epoch : 01 step : 0007 loss = 0.482453 accuracy= 0.781250
epoch : 01 step : 0008 loss = 0.657392 accuracy= 0.750000
epoch : 01 step : 0009 loss = 0.604300 accuracy= 0.781250
epoch : 01 step : 0010 loss = 0.575462 accuracy= 0.750000
epoch : 01 step : 0011 loss = 0.438139 accuracy= 0.875000
epoch : 01 step : 0012 loss = 0.534019 accuracy= 0.812500
epoch : 01 step : 0013 loss = 0.735172 accuracy= 0.687500
epoch : 01 step : 0014 loss = 0.428723 accuracy= 0.875000
epoch : 01 step : 0015 loss = 0.355932 accuracy= 0.843750
epoch : 01 step : 0016 loss = 0.377038 accuracy= 0.875000
epoch : 01 step : 0017 loss = 0.781001 accuracy= 0.687500
epoch : 01 ste

epoch : 01 step : 0142 loss = 0.468693 accuracy= 0.781250
epoch : 01 step : 0143 loss = 0.473246 accuracy= 0.875000
epoch : 01 step : 0144 loss = 0.799610 accuracy= 0.562500
epoch : 01 step : 0145 loss = 0.863463 accuracy= 0.656250
epoch : 01 step : 0146 loss = 0.497872 accuracy= 0.781250
epoch : 01 step : 0147 loss = 0.740458 accuracy= 0.750000
epoch : 01 step : 0148 loss = 0.447085 accuracy= 0.812500
epoch : 01 step : 0149 loss = 0.600801 accuracy= 0.812500
epoch : 01 step : 0150 loss = 0.471183 accuracy= 0.750000
epoch : 01 step : 0151 loss = 0.758767 accuracy= 0.687500
epoch : 01 step : 0152 loss = 0.551732 accuracy= 0.781250
epoch : 01 step : 0153 loss = 0.594437 accuracy= 0.781250
epoch : 01 step : 0154 loss = 0.457374 accuracy= 0.875000
epoch : 01 step : 0155 loss = 0.714646 accuracy= 0.687500
epoch : 01 step : 0156 loss = 0.552681 accuracy= 0.718750
epoch : 01 step : 0157 loss = 0.474276 accuracy= 0.812500
epoch : 01 step : 0158 loss = 0.473254 accuracy= 0.875000
epoch : 01 ste


<Train> Loss = 0.575084 Accuracy = 0.798224
<Test> Loss = 0.701805 Accuracy = 0.743960


epoch : 02 step : 0001 loss = 0.355329 accuracy= 0.968750
epoch : 02 step : 0002 loss = 0.331769 accuracy= 0.875000
epoch : 02 step : 0003 loss = 0.327741 accuracy= 0.937500
epoch : 02 step : 0004 loss = 0.368628 accuracy= 0.843750
epoch : 02 step : 0005 loss = 0.573180 accuracy= 0.843750
epoch : 02 step : 0006 loss = 0.245284 accuracy= 1.000000
epoch : 02 step : 0007 loss = 0.385914 accuracy= 0.875000
epoch : 02 step : 0008 loss = 0.462163 accuracy= 0.906250
epoch : 02 step : 0009 loss = 0.443493 accuracy= 0.843750
epoch : 02 step : 0010 loss = 0.420187 accuracy= 0.906250
epoch : 02 step : 0011 loss = 0.280360 accuracy= 0.937500
epoch : 02 step : 0012 loss = 0.412240 accuracy= 0.875000
epoch : 02 step : 0013 loss = 0.330522 accuracy= 0.937500
epoch : 02 step : 0014 loss = 0.292678 accuracy= 0.937500
epoch : 02 step : 0015 loss = 0.230655 accuracy= 1.000000
epoch : 02 step : 0016 loss = 0.347571 accuracy= 0.906250
epoch : 02 step : 0017 loss = 0.398070 accuracy= 0.843750
epoch : 02 ste

epoch : 02 step : 0142 loss = 0.196729 accuracy= 0.968750
epoch : 02 step : 0143 loss = 0.266993 accuracy= 0.937500
epoch : 02 step : 0144 loss = 0.400001 accuracy= 0.843750
epoch : 02 step : 0145 loss = 0.469618 accuracy= 0.843750
epoch : 02 step : 0146 loss = 0.552193 accuracy= 0.906250
epoch : 02 step : 0147 loss = 0.329677 accuracy= 0.875000
epoch : 02 step : 0148 loss = 0.437320 accuracy= 0.906250
epoch : 02 step : 0149 loss = 0.351112 accuracy= 0.843750
epoch : 02 step : 0150 loss = 0.202323 accuracy= 0.968750
epoch : 02 step : 0151 loss = 0.326979 accuracy= 0.875000
epoch : 02 step : 0152 loss = 0.344731 accuracy= 0.875000
epoch : 02 step : 0153 loss = 0.404825 accuracy= 0.843750
epoch : 02 step : 0154 loss = 0.353390 accuracy= 0.875000
epoch : 02 step : 0155 loss = 0.308420 accuracy= 0.875000
epoch : 02 step : 0156 loss = 0.399093 accuracy= 0.812500
epoch : 02 step : 0157 loss = 0.419452 accuracy= 0.875000
epoch : 02 step : 0158 loss = 0.524132 accuracy= 0.843750
epoch : 02 ste


<Train> Loss = 0.368234 Accuracy = 0.892648
<Test> Loss = 0.737128 Accuracy = 0.731092


epoch : 03 step : 0001 loss = 0.258760 accuracy= 0.968750
epoch : 03 step : 0002 loss = 0.290477 accuracy= 0.937500
epoch : 03 step : 0003 loss = 0.275060 accuracy= 0.906250
epoch : 03 step : 0004 loss = 0.196970 accuracy= 0.968750
epoch : 03 step : 0005 loss = 0.185229 accuracy= 0.968750
epoch : 03 step : 0006 loss = 0.250106 accuracy= 0.937500
epoch : 03 step : 0007 loss = 0.162380 accuracy= 0.968750
epoch : 03 step : 0008 loss = 0.224377 accuracy= 0.906250
epoch : 03 step : 0009 loss = 0.369155 accuracy= 0.875000
epoch : 03 step : 0010 loss = 0.133110 accuracy= 1.000000
epoch : 03 step : 0011 loss = 0.184175 accuracy= 0.968750
epoch : 03 step : 0012 loss = 0.214982 accuracy= 0.968750
epoch : 03 step : 0013 loss = 0.297886 accuracy= 0.906250
epoch : 03 step : 0014 loss = 0.233773 accuracy= 0.968750
epoch : 03 step : 0015 loss = 0.178072 accuracy= 1.000000
epoch : 03 step : 0016 loss = 0.247383 accuracy= 0.937500
epoch : 03 step : 0017 loss = 0.235792 accuracy= 0.937500
epoch : 03 ste

epoch : 03 step : 0142 loss = 0.282956 accuracy= 0.906250
epoch : 03 step : 0143 loss = 0.274502 accuracy= 0.937500
epoch : 03 step : 0144 loss = 0.292415 accuracy= 0.937500
epoch : 03 step : 0145 loss = 0.275087 accuracy= 0.906250
epoch : 03 step : 0146 loss = 0.192209 accuracy= 0.968750
epoch : 03 step : 0147 loss = 0.160145 accuracy= 0.968750
epoch : 03 step : 0148 loss = 0.311731 accuracy= 0.875000
epoch : 03 step : 0149 loss = 0.237675 accuracy= 0.937500
epoch : 03 step : 0150 loss = 0.243976 accuracy= 0.937500
epoch : 03 step : 0151 loss = 0.165839 accuracy= 0.968750
epoch : 03 step : 0152 loss = 0.247993 accuracy= 0.968750
epoch : 03 step : 0153 loss = 0.170294 accuracy= 0.968750
epoch : 03 step : 0154 loss = 0.373578 accuracy= 0.875000
epoch : 03 step : 0155 loss = 0.121212 accuracy= 1.000000
epoch : 03 step : 0156 loss = 0.184839 accuracy= 0.968750
epoch : 03 step : 0157 loss = 0.245050 accuracy= 0.906250
epoch : 03 step : 0158 loss = 0.136165 accuracy= 1.000000
epoch : 03 ste


<Train> Loss = 0.245462 Accuracy = 0.936938
<Test> Loss = 0.806719 Accuracy = 0.707195


In [27]:
train_loss = pd.DataFrame({"train_loss":train_loss})
train_acc = pd.DataFrame({"train_acc":train_acc})
test_loss = pd.DataFrame({"test_loss":test_loss})
test_acc = pd.DataFrame({"test_acc":test_acc})
df = pd.concat([train_loss,train_acc,test_loss,test_acc], axis = 1)
df.to_csv("./BiLSTM_mecab_morphs_eqaul_1/loss_accuracy2.csv", sep =",", index=False)

train_writer.close()
duration = time.time() - start_time
minute = int(duration / 60)
second = int(duration) % 60
print("%dminutes %dseconds" % (minute,second))
save_path = saver.save(sess, modelName)

print ('save_path',save_path)

44minutes 43seconds
save_path BiLSTM_mecab_morphs_eqaul_1/BiLSTM_model.ckpt


# 에러나는 거 지우기..

In [40]:
lis = []
for i in range(len(data)):
    if len(data[i]) == 0:
        lis.append(i)

In [48]:
df.index

Int64Index([19600,  2370, 11985, 26165, 19178, 26594, 27716, 20978, 23177,
             8799,
            ...
            13445, 11016,  5500, 16705, 16156,  7840, 20908, 12080, 28343,
            24066],
           dtype='int64', length=28798)

In [51]:
bugidx = []
for i in lis:
    bugidx.append(df.index[i])

In [52]:
df.drop(bugidx,inplace=True)

In [53]:
df.head()

,category,c_origin,jss,label
19600,재무·금융,재무·세무·IR,[맡은 일은 끝까지 완벽하게]\n고등학교 생활은 제가 책임감 있는 사람으로 성장하도...,1
2370,생산·물류,생산관리·품질관리,"스터디센터, OOOO, 초콜릿 공장 등에서 아르바이트를 경험했습니다. 각 직무마다 ...",2
11985,재무·금융,재무회계,"[단골손님]\n저는 고객이란 '신뢰를 바탕으로 한 잠재적이며, 지속적인 인간관계' ...",1
26165,영업·마케팅,마케팅·광고·분석,[내가 치킨을 사먹는다는다면?]\n\n지난 학기 때 경제이론을 실제 사례로 설명하는...,3
19178,서버·시스템,시스템프로그래머,[기본에 충실한 대인관계의 달인]\n매 학기 근로장학생을 해서 찾아간 분들에게 좋은...,6
